In [1]:
%jsroot on

In [2]:
struct x_values{
    double xpos[2];
    double xneg[2];        
};

In [3]:
x_values isocrony_x(Double_t* params_1, const Double_t* err_params_1, Double_t* params_2, const Double_t* err_params_2){
    x_values x;

    // non usare valori assoluti per a, b e c per evitare di ottenere valore di isocronia errato.
    double a = params_1[0]-params_2[0], err_a = abs(err_params_1[0])+abs(err_params_2[0]);
    double b = params_1[1]-params_2[1], err_b = abs(err_params_1[1])+abs(err_params_2[1]);
    double c = params_1[2]-params_2[2], err_c = abs(err_params_1[2])+abs(err_params_2[2]);

    double delta = pow(b, 2) - 4*a*c;

    if(delta<0){
        std::cout << "delta is negative!" << std::endl;
        return {{0,0}, {0,0}};
    }

    x.xpos[0] = (-b+sqrt(delta))/(2*a);
    x.xneg[0] = (-b-sqrt(delta))/(2*a);

    double dda_sqr_pos = pow(((-1)*c/(a*sqrt(delta))) - ((-b + sqrt(delta))/(2*a*a)), 2);
    double ddb_sqr_pos = pow((-1 + b/sqrt(delta))/(2*a), 2);
    double ddc_sqr_pos = pow(-1/sqrt(delta), 2);

    x.xpos[1] = sqrt((dda_sqr_pos*err_a*err_a) + (ddb_sqr_pos*err_b*err_b) + (ddc_sqr_pos*err_c*err_c));

    double dda_sqr_neg = pow((b*b - 2*a*c + b*sqrt(delta))/(2*a*a*sqrt(delta)), 2);
    double ddb_sqr_neg = pow((-1 - (b/sqrt(delta)))/(2*a), 2);
    double ddc_sqr_neg = pow(1/sqrt(delta), 2);

    x.xneg[1] = sqrt((dda_sqr_neg*err_a*err_a) + (ddb_sqr_neg*err_b*err_b) + (ddc_sqr_neg*err_c*err_c));

    return x;
}

In [4]:
double* get_isoX(x_values x, TGraphErrors _g){
    
    double* _x = new double[2];

    if(x.xpos[0]>TMath::MinElement(_g.GetN(), _g.GetX())){
        _x[0] = x.xpos[0]; _x[1] = x.xpos[1];
    }else{
        _x[0] = x.xneg[0]; _x[1] = x.xneg[1];
    }
    
    return _x;

}

In [5]:
double get_err_T(double x_iso, TGraphErrors g1, TGraphErrors g2){

    double Terr_iso_1 = 0;
    double Terr_iso_2 = 0;
    double range = abs(g1.GetX()[0]-x_iso);

    for(int i=0; i<g1.GetN(); i++){
        // std::cout << i << " " << abs(x_iso-g1.GetX()[i]) << " x: " << g1.GetX()[i] << std::endl;
        // algorimo per identificazione dei valori di T1 T2 più
        // vicini al valore di isocronia

        if((abs(x_iso-g1.GetX()[i])<range)){
            Terr_iso_1 = g1.GetErrorY(i);
            Terr_iso_2 = g2.GetErrorY(i);
            range = abs(x_iso-g1.GetX()[i]);
        }
    }

    return sqrt(pow(Terr_iso_1, 2)+pow(Terr_iso_2, 2));
}

In [6]:
TCanvas* c1 = new TCanvas("c", "", 600, 500);
c1->SetMargin(0.16, 0.06, 0.12, 0.06);

TGraphErrors* g1 = new TGraphErrors("../dati/computed_T1_x.txt"); g1->SetName("g1");
g1->SetMarkerStyle(4);
g1->SetLineColor(kBlack);
TGraphErrors* g2 = new TGraphErrors("../dati/computed_T2_x.txt"); g2->SetName("g2");
g2->SetLineColor(kRed);
g2->SetMarkerStyle(4);
g2->SetMarkerColor(kRed);

TF1* f1 = new TF1("f1", "([0]*x*x)+([1]*x)+[2]"); // a1 = p0, b1 = p1, c1 = p2;
f1->SetLineColor(kBlack);
// f1->SetParameters(0.0001, 0.001, 1.4);
TF1* f2 = new TF1("f2", "([0]*x*x)+([1]*x)+[2]"); // a2 = p0, b2 = p1, c2 = p2;
f2->SetParameters(2, -1, 0.8);
f2->SetLineColor(kRed);

g1->Draw("ap");
g1->Fit("f1", "V");

g2->Draw("p");
g2->Fit("f2", "V");


TLegend* leg = new TLegend(0.25, 0.65, 0.5, 0.8);
leg->AddEntry("g1", "Periodi T1");
leg->AddEntry("g2", "Periodi T2");
leg->SetBorderSize(0);
leg->Draw();

c1->Draw();

 **********
 **    1 **SET PRINT           2
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 p0           0.00000e+00  3.00000e-01     no limits
     2 p1           0.00000e+00  3.00000e-01     no limits
     3 p2           0.00000e+00  3.00000e-01     no limits
 **********
 **    3 **SET ERR           1
 **********
 **********
 **    4 **SET PRINT           2
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1345        0.01
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-05
 FCN=3.68396e+07 FROM MIGRAD    STATUS=INITIATE       31 CALLS          32 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUE

Info in <TMinuitMinimizer::Minimize>: Finished to run MIGRAD - status 0
Info in <TMinuitMinimizer::Minimize>: Finished to run MIGRAD - status 0
